In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import glob

# Gauge Data by HUC10 Watershed

In [14]:
input_gauge_watersheds = r'../Data/stream_gauges/KentuckyStreamGaugeLocations_Watersheds.csv'

df_gauges = pd.read_csv(input_gauge_watersheds, 
                        dtype={'site_no':'string', 'huc10':'string'}, 
                        parse_dates=['gh_start', 'gh_end', 'sf_start', 'sf_end'])

df_gauges.drop(columns=['Field1', 'huc12', 'huc12_name', 'huc12_areasqkm'], inplace=True)

df_gauges = df_gauges.astype({'station_nm':'string', 'datum':'string', 'huc10_name':'string'})

df_gauges.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275 entries, 0 to 274
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   site_no         275 non-null    string        
 1   station_nm      275 non-null    string        
 2   lat             275 non-null    float64       
 3   long            275 non-null    float64       
 4   datum           275 non-null    string        
 5   altitude        275 non-null    float64       
 6   gh_ind          275 non-null    int64         
 7   gh_start        232 non-null    datetime64[ns]
 8   gh_end          232 non-null    datetime64[ns]
 9   sf_ind          275 non-null    int64         
 10  sf_start        215 non-null    datetime64[ns]
 11  sf_end          215 non-null    datetime64[ns]
 12  huc10           275 non-null    string        
 13  huc10_name      275 non-null    string        
 14  huc10_areasqkm  275 non-null    float64       
dtypes: dat

In [15]:
df_gauges.head()

,site_no,station_nm,lat,long,datum,altitude,gh_ind,gh_start,gh_end,sf_ind,sf_start,sf_end,huc10,huc10_name,huc10_areasqkm
0,3207965,"GRAPEVINE CREEK NEAR PHYLLIS, KY",37.432606,-82.353760,NAD83,797.19,1,2007-10-01,2024-02-15,1,1989-04-25,2024-02-16,0507020202,Fishtrap Lake-Levisa Fork,418.119995
1,3208000,"LEVISA FORK BELOW FISHTRAP DAM NEAR MILLARD, KY",37.425938,-82.412370,NAD83,599.16,1,2008-07-31,2024-02-15,0,NaT,NaT,0507020202,Fishtrap Lake-Levisa Fork,418.119995
2,3209300,"RUSSELL FORK AT ELKHORN CITY, KY",37.303997,-82.342926,NAD83,773.00,1,2008-10-10,2017-10-03,0,NaT,NaT,0507020207,Russell Fork-Levisa Fork,449.799988
3,3209310,"ELKHORN LAKE AT JENKINS, KY",37.168354,-82.635060,NAD83,1573.06,1,2023-12-07,2024-02-15,0,NaT,NaT,0507020207,Russell Fork-Levisa Fork,449.799988
4,3209325,"ELKHORN CREEK AT BURDINE, KY",37.187830,-82.604570,NAD83,0.00,1,2024-01-31,2024-02-15,0,NaT,NaT,0507020207,Russell Fork-Levisa Fork,449.799988


## Plot all data by Watershed

In [57]:
watersheds = df_gauges['huc10'].unique()

In [1]:
for watershed in watersheds:
    
    fig, ax1 = plt.subplots(figsize=(7,7))
    
    ax2 = ax1.twinx()

    gauges = df_gauges.loc[df_gauges['huc10'] == watershed]

    for index, row in gauges.iterrows():

        gauge = row['site_no']

        if row['gh_ind'] == 1:

            gaugeheight_path = glob.glob(f'../Datasets/stream_gauges/GageHeight/*{gauge}*.csv')[0]
    
            df_gh = pd.read_csv(gaugeheight_path, parse_dates=['datetime'])

            df_gh.drop(index=0, inplace=True)

            ax1.plot(df_gh['datetime'], df_gh.iloc[:,4], label=f'{gauge}, flood stage')

        if row['sf_ind'] == 1:

            streamflow_path = glob.glob(f'../Datasets/stream_gauges/Streamflow/*{gauge}*.csv')[0]

            df_sf = pd.read_csv(streamflow_path, parse_dates=['datetime'])

            df_sf.drop(index=0, inplace=True)

            ax2.plot(df_sf['datetime'], df_sf.iloc[:,4], label=f'{gauge}, discharge')

    ax1.set_xlabel('Date')
    
    ax1.set_ylabel('Flood Stage (ft)')

    ax2.set_ylabel('Discharge (m3/s)')

    ax1.set_title(f'Stream Gauge Data for HUC10 Waterhed {watershed}')

    plt.show()

    break

    

NameError: name 'watersheds' is not defined

# Identify Elevated Stream Flow Events

# Calculate Elevated Stream Flow Frequency